# Corpora and vector spaces
https://radimrehurek.com/gensim/tut1.html

In [13]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
import os
import tempfile
TEMP_FOLDER = tempfile.gettempdir()
print 'Folder "{}" will be used to save temporary dictionary and corpus.'.format(TEMP_FOLDER)

from gensim import corpora, models, similarities

Folder "/var/folders/pv/r00j5j0n5g99lmw66v7rhlqc0000gn/T" will be used to save temporary dictionary and corpus.


In [2]:
documents = ["Human machine interface for lab abc computer applications",
             "A survey of user opinion of computer system response time",
             "The EPS user interface management system",
             "System and human system engineering testing of EPS",              
             "Relation of user perceived response time to error measurement",
             "The generation of random binary unordered trees",
             "The intersection graph of paths in trees",
             "Graph minors IV Widths of trees and well quasi ordering",
             "Graph minors A survey"]

In [3]:
# remove common words and tokenize
stoplist = set('for a of the and to in'.split())
texts = [[word for word in document.lower().split() if word not in stoplist] for document in documents]

# lemmatization - get original form of words ????

# remove words that appear only once
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1
        
texts = [[token for token in text if frequency[token] > 1] for text in texts]
from pprint import pprint
pprint(texts)

[['human', 'interface', 'computer'],
 ['survey', 'user', 'computer', 'system', 'response', 'time'],
 ['eps', 'user', 'interface', 'system'],
 ['system', 'human', 'system', 'eps'],
 ['user', 'response', 'time'],
 ['trees'],
 ['graph', 'trees'],
 ['graph', 'minors', 'trees'],
 ['graph', 'minors', 'survey']]


In [4]:
dictionary = corpora.Dictionary(texts)
dictionary.save(os.path.join(TEMP_FOLDER, 'deerwester.dict'))
print dictionary

2017-07-28 11:10:36,821 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2017-07-28 11:10:36,823 : INFO : built Dictionary(12 unique tokens: [u'minors', u'graph', u'system', u'trees', u'eps']...) from 9 documents (total 29 corpus positions)
2017-07-28 11:10:36,825 : INFO : saving Dictionary object under /var/folders/pv/r00j5j0n5g99lmw66v7rhlqc0000gn/T/deerwester.dict, separately None
2017-07-28 11:10:36,829 : INFO : saved /var/folders/pv/r00j5j0n5g99lmw66v7rhlqc0000gn/T/deerwester.dict


Dictionary(12 unique tokens: [u'minors', u'graph', u'system', u'trees', u'eps']...)


In [5]:
print dictionary.token2id

{u'minors': 11, u'graph': 10, u'system': 6, u'trees': 9, u'eps': 8, u'computer': 1, u'survey': 5, u'user': 7, u'human': 2, u'time': 4, u'interface': 0, u'response': 3}


In [6]:
# bag of words
corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize(os.path.join(TEMP_FOLDER, 'deerwester.mm'), corpus)
corpus

2017-07-28 11:13:14,267 : INFO : storing corpus in Matrix Market format to /var/folders/pv/r00j5j0n5g99lmw66v7rhlqc0000gn/T/deerwester.mm
2017-07-28 11:13:14,270 : INFO : saving sparse matrix to /var/folders/pv/r00j5j0n5g99lmw66v7rhlqc0000gn/T/deerwester.mm
2017-07-28 11:13:14,272 : INFO : PROGRESS: saving document #0
2017-07-28 11:13:14,274 : INFO : saved 9x12 matrix, density=25.926% (28/108)
2017-07-28 11:13:14,277 : INFO : saving MmCorpus index to /var/folders/pv/r00j5j0n5g99lmw66v7rhlqc0000gn/T/deerwester.mm.index


[[(0, 1), (1, 1), (2, 1)],
 [(1, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)],
 [(0, 1), (6, 1), (7, 1), (8, 1)],
 [(2, 1), (6, 2), (8, 1)],
 [(3, 1), (4, 1), (7, 1)],
 [(9, 1)],
 [(9, 1), (10, 1)],
 [(9, 1), (10, 1), (11, 1)],
 [(5, 1), (10, 1), (11, 1)]]

In [7]:
corpus = corpora.MmCorpus(os.path.join(TEMP_FOLDER, 'deerwester.mm'))

2017-07-28 11:13:43,272 : INFO : loaded corpus index from /var/folders/pv/r00j5j0n5g99lmw66v7rhlqc0000gn/T/deerwester.mm.index
2017-07-28 11:13:43,274 : INFO : initializing corpus reader from /var/folders/pv/r00j5j0n5g99lmw66v7rhlqc0000gn/T/deerwester.mm
2017-07-28 11:13:43,275 : INFO : accepted corpus with 9 documents, 12 features, 28 non-zero entries


# Topics and transformations
https://radimrehurek.com/gensim/tut2.html

In [8]:
tfidf = models.TfidfModel(corpus)

2017-07-28 11:13:47,055 : INFO : collecting document frequencies
2017-07-28 11:13:47,057 : INFO : PROGRESS: processing document #0
2017-07-28 11:13:47,061 : INFO : calculating IDF weights for 9 documents and 11 features (28 matrix non-zeros)


In [10]:
corpus_tfidf = tfidf[corpus]
for i in corpus_tfidf:
    print i

[(0, 0.5773502691896257), (1, 0.5773502691896257), (2, 0.5773502691896257)]
[(1, 0.44424552527467476), (3, 0.44424552527467476), (4, 0.44424552527467476), (5, 0.44424552527467476), (6, 0.3244870206138555), (7, 0.3244870206138555)]
[(0, 0.5710059809418182), (6, 0.4170757362022777), (7, 0.4170757362022777), (8, 0.5710059809418182)]
[(2, 0.49182558987264147), (6, 0.7184811607083769), (8, 0.49182558987264147)]
[(3, 0.6282580468670046), (4, 0.6282580468670046), (7, 0.45889394536615247)]
[(9, 1.0)]
[(9, 0.7071067811865475), (10, 0.7071067811865475)]
[(9, 0.5080429008916749), (10, 0.5080429008916749), (11, 0.695546419520037)]
[(5, 0.6282580468670046), (10, 0.45889394536615247), (11, 0.6282580468670046)]


In [14]:
lsi = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=2)
corpus_lsi = lsi[corpus_tfidf] # create a double wrapper over the original corpus: bow->tfidf->fold-in-lsi

2017-07-28 11:18:58,400 : INFO : using serial LSI version on this node
2017-07-28 11:18:58,402 : INFO : updating model with new documents
2017-07-28 11:18:58,405 : INFO : preparing a new chunk of documents
2017-07-28 11:18:58,409 : INFO : using 100 extra samples and 2 power iterations
2017-07-28 11:18:58,410 : INFO : 1st phase: constructing (12, 102) action matrix
2017-07-28 11:18:58,414 : INFO : orthonormalizing (12, 102) action matrix
2017-07-28 11:18:58,423 : INFO : 2nd phase: running dense svd on (12, 9) matrix
2017-07-28 11:18:58,428 : INFO : computing the final decomposition
2017-07-28 11:18:58,430 : INFO : keeping 2 factors (discarding 47.565% of energy spectrum)
2017-07-28 11:18:58,431 : INFO : processed documents up to #9
2017-07-28 11:18:58,437 : INFO : topic #0(1.594): 0.703*"trees" + 0.538*"graph" + 0.402*"minors" + 0.187*"survey" + 0.061*"system" + 0.060*"response" + 0.060*"time" + 0.058*"user" + 0.049*"computer" + 0.035*"interface"
2017-07-28 11:18:58,439 : INFO : topic #

In [21]:
lsi.print_topics(-1)

2017-07-28 11:21:14,638 : INFO : topic #0(1.594): 0.703*"trees" + 0.538*"graph" + 0.402*"minors" + 0.187*"survey" + 0.061*"system" + 0.060*"response" + 0.060*"time" + 0.058*"user" + 0.049*"computer" + 0.035*"interface"
2017-07-28 11:21:14,640 : INFO : topic #1(1.476): -0.460*"system" + -0.373*"user" + -0.332*"eps" + -0.328*"interface" + -0.320*"response" + -0.320*"time" + -0.293*"computer" + -0.280*"human" + -0.171*"survey" + 0.161*"trees"


[(0,
  u'0.703*"trees" + 0.538*"graph" + 0.402*"minors" + 0.187*"survey" + 0.061*"system" + 0.060*"response" + 0.060*"time" + 0.058*"user" + 0.049*"computer" + 0.035*"interface"'),
 (1,
  u'-0.460*"system" + -0.373*"user" + -0.332*"eps" + -0.328*"interface" + -0.320*"response" + -0.320*"time" + -0.293*"computer" + -0.280*"human" + -0.171*"survey" + 0.161*"trees"')]

In [22]:
for doc in corpus_lsi:
    print doc

[(0, 0.066007833960902179), (1, -0.52007033063618469)]
[(0, 0.19667592859142241), (1, -0.76095631677000575)]
[(0, 0.089926399724462175), (1, -0.72418606267525076)]
[(0, 0.075858476521780099), (1, -0.63205515860034245)]
[(0, 0.10150299184979919), (1, -0.57373084830029641)]
[(0, 0.70321089393783176), (1, 0.16115180214025568)]
[(0, 0.8774787673119836), (1, 0.16758906864659112)]
[(0, 0.90986246868185816), (1, 0.14086553628718695)]
[(0, 0.61658253505692784), (1, -0.053929075663896098)]


In [23]:
# lsi.save('/tmp/model.lsi') # same for tfidf, lda, ...
# lsi = models.LsiModel.load('/tmp/model.lsi')

In [25]:
lda = models.LdaModel(corpus, id2word=dictionary, num_topics=2)
corpus_lda = lda[corpus] # create a double wrapper over the original corpus: bow->tfidf->fold-in-lsi

2017-07-28 11:41:17,504 : INFO : using symmetric alpha at 0.5
2017-07-28 11:41:17,506 : INFO : using symmetric eta at 0.0833333333333
2017-07-28 11:41:17,508 : INFO : using serial LDA version on this node
2017-07-28 11:41:17,510 : INFO : running online (single-pass) LDA training, 2 topics, 1 passes over the supplied corpus of 9 documents, updating model once every 9 documents, evaluating perplexity every 9 documents, iterating 50x with a convergence threshold of 0.001000
2017-07-28 11:41:17,512 : WARNING : too few updates, training might not converge; consider increasing the number of passes or iterations to improve accuracy
2017-07-28 11:41:17,540 : INFO : -3.293 per-word bound, 9.8 perplexity estimate based on a held-out corpus of 9 documents with 29 words
2017-07-28 11:41:17,541 : INFO : PROGRESS: pass 0, at document #9/9
2017-07-28 11:41:17,562 : INFO : topic #0 (0.500): 0.144*"graph" + 0.133*"trees" + 0.128*"system" + 0.099*"minors" + 0.090*"survey" + 0.080*"user" + 0.071*"eps" + 

In [26]:
lda.print_topics(-1)

2017-07-28 11:41:29,161 : INFO : topic #0 (0.500): 0.144*"graph" + 0.133*"trees" + 0.128*"system" + 0.099*"minors" + 0.090*"survey" + 0.080*"user" + 0.071*"eps" + 0.059*"time" + 0.053*"interface" + 0.051*"computer"
2017-07-28 11:41:29,164 : INFO : topic #1 (0.500): 0.119*"user" + 0.115*"system" + 0.106*"human" + 0.103*"response" + 0.099*"computer" + 0.097*"interface" + 0.090*"time" + 0.075*"eps" + 0.056*"trees" + 0.054*"survey"


[(0,
  u'0.144*"graph" + 0.133*"trees" + 0.128*"system" + 0.099*"minors" + 0.090*"survey" + 0.080*"user" + 0.071*"eps" + 0.059*"time" + 0.053*"interface" + 0.051*"computer"'),
 (1,
  u'0.119*"user" + 0.115*"system" + 0.106*"human" + 0.103*"response" + 0.099*"computer" + 0.097*"interface" + 0.090*"time" + 0.075*"eps" + 0.056*"trees" + 0.054*"survey"')]

In [27]:
for doc in corpus_lda:
    print doc

[(0, 0.14452595506523735), (1, 0.85547404493476265)]
[(0, 0.13375730937679836), (1, 0.86624269062320169)]
[(0, 0.16280797983646775), (1, 0.83719202016353234)]
[(0, 0.20162443403887584), (1, 0.79837556596112413)]
[(0, 0.15464291926117757), (1, 0.84535708073882232)]
[(0, 0.72522945129831251), (1, 0.27477054870168743)]
[(0, 0.82061413177823528), (1, 0.17938586822176472)]
[(0, 0.86477908146734084), (1, 0.13522091853265922)]
[(0, 0.86208799187064844), (1, 0.13791200812935162)]


# Similarity
https://radimrehurek.com/gensim/tut3.html

In [29]:
dictionary = corpora.Dictionary.load(os.path.join(TEMP_FOLDER, 'deerwester.dict'))
corpus = corpora.MmCorpus(os.path.join(TEMP_FOLDER, 'deerwester.mm'))

2017-07-28 11:47:01,660 : INFO : loading Dictionary object from /var/folders/pv/r00j5j0n5g99lmw66v7rhlqc0000gn/T/deerwester.dict
2017-07-28 11:47:01,662 : INFO : loaded /var/folders/pv/r00j5j0n5g99lmw66v7rhlqc0000gn/T/deerwester.dict
2017-07-28 11:47:01,664 : INFO : loaded corpus index from /var/folders/pv/r00j5j0n5g99lmw66v7rhlqc0000gn/T/deerwester.mm.index
2017-07-28 11:47:01,666 : INFO : initializing corpus reader from /var/folders/pv/r00j5j0n5g99lmw66v7rhlqc0000gn/T/deerwester.mm
2017-07-28 11:47:01,668 : INFO : accepted corpus with 9 documents, 12 features, 28 non-zero entries


In [30]:
lsi = models.LsiModel(corpus, id2word=dictionary, num_topics=2)

2017-07-28 11:47:21,126 : INFO : using serial LSI version on this node
2017-07-28 11:47:21,128 : INFO : updating model with new documents
2017-07-28 11:47:21,130 : INFO : preparing a new chunk of documents
2017-07-28 11:47:21,132 : INFO : using 100 extra samples and 2 power iterations
2017-07-28 11:47:21,133 : INFO : 1st phase: constructing (12, 102) action matrix
2017-07-28 11:47:21,134 : INFO : orthonormalizing (12, 102) action matrix
2017-07-28 11:47:21,137 : INFO : 2nd phase: running dense svd on (12, 9) matrix
2017-07-28 11:47:21,139 : INFO : computing the final decomposition
2017-07-28 11:47:21,141 : INFO : keeping 2 factors (discarding 43.156% of energy spectrum)
2017-07-28 11:47:21,142 : INFO : processed documents up to #9
2017-07-28 11:47:21,144 : INFO : topic #0(3.341): 0.644*"system" + 0.404*"user" + 0.301*"eps" + 0.265*"time" + 0.265*"response" + 0.240*"computer" + 0.221*"human" + 0.206*"survey" + 0.198*"interface" + 0.036*"graph"
2017-07-28 11:47:21,145 : INFO : topic #1(2

In [31]:
doc = "Human computer interaction"
vec_bow = dictionary.doc2bow(doc.lower().split())
vec_lsi = lsi[vec_bow]
print vec_lsi, vec_bow

[(0, 0.46182100453271624), (1, 0.070027665279001478)] [(1, 1), (2, 1)]


In [32]:
index = similarities.MatrixSimilarity(lsi[corpus])

2017-07-28 12:00:03,454 : WARNING : scanning corpus to determine the number of features (consider setting `num_features` explicitly)
2017-07-28 12:00:03,458 : INFO : creating matrix with 9 documents and 2 features


In [35]:
index.save(os.path.join(TEMP_FOLDER, 'deerwester.index'))
index = similarities.MatrixSimilarity.load(os.path.join(TEMP_FOLDER, 'deerwester.index'))

2017-07-28 12:00:50,739 : INFO : saving MatrixSimilarity object under /var/folders/pv/r00j5j0n5g99lmw66v7rhlqc0000gn/T/deerwester.index, separately None
2017-07-28 12:00:50,742 : INFO : saved /var/folders/pv/r00j5j0n5g99lmw66v7rhlqc0000gn/T/deerwester.index
2017-07-28 12:00:50,744 : INFO : loading MatrixSimilarity object from /var/folders/pv/r00j5j0n5g99lmw66v7rhlqc0000gn/T/deerwester.index
2017-07-28 12:00:50,746 : INFO : loaded /var/folders/pv/r00j5j0n5g99lmw66v7rhlqc0000gn/T/deerwester.index


In [37]:
sims = index[vec_lsi]
print list(enumerate(sims))

[(0, 0.99809301), (1, 0.93748635), (2, 0.99844527), (3, 0.9865886), (4, 0.90755945), (5, -0.12416792), (6, -0.10639259), (7, -0.098794632), (8, 0.050041769)]


In [41]:
texts

[['human', 'interface', 'computer'],
 ['survey', 'user', 'computer', 'system', 'response', 'time'],
 ['eps', 'user', 'interface', 'system'],
 ['system', 'human', 'system', 'eps'],
 ['user', 'response', 'time'],
 ['trees'],
 ['graph', 'trees'],
 ['graph', 'minors', 'trees'],
 ['graph', 'minors', 'survey']]